In [ ]:
# =============================== #
#   Multi-Cell / Per-Cell Tilt    #
#   Sionna 1.1.0 (no ray tracing) #
# =============================== #

# (optional) avoid TF pre-grabbing VRAM (restart kernel after toggling)
# import os
# os.environ["TF_GPU_ALLOCATOR"] = "cuda_malloc_async"
# os.environ["TF_FORCE_GPU_ALLOW_GROWTH"] = "true"

import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt

from sionna.phy.channel.tr38901 import PanelArray, UMa
from sionna.phy.ofdm import ResourceGrid
from sionna.phy.channel import OFDMChannel

import os
import sys
import random
import math
sys.path.append('../..')

from cns_sionna_sim import MultiCellSim
from sionna_sim_helpers import add_site_with_dualband_cells, iter_clustered_sites, rsrp_rows_as_dicts

np.set_printoptions(suppress=True, precision=6)

In [ ]:
# Defaults (8x1) - used if no antenna params specified
sim = MultiCellSim(bs_rows=8, bs_cols=1, fft_size=32)
sim.configure_naming(use_site='id', sector_mode='1based', pattern='{band}{site}{sector}')

n_sites = 10
for idx, (x, y, az0) in enumerate(iter_clustered_sites(n_sites, spacing=500, center=(0.0, 0.0), jitter=0.06, seed=7), start=1):
    site_name = f"SITE{idx:04d}A"
    
    add_site_with_dualband_cells(
        sim,
        site_name=site_name,
        x=x, y=y,
        height_m=20.0,
        az0_deg=az0,
        # High band: 2.5 GHz with 8x1 array (narrow beam for capacity)
        fc_hi_hz=2500e6, band_hi="H", tilt_hi_deg=9.0, pwr_hi_dbm=0.0,
        bs_rows_hi=4, bs_cols_hi=2,      # ← High band antenna
        antenna_pattern_hi='38.901',
        # Low band: 600 MHz with 4x4 array (wider beam for coverage)
        fc_lo_hz=600e6, band_lo="L", tilt_lo_deg=9.0, pwr_lo_dbm=0.0,
        bs_rows_lo=8, bs_cols_lo=1,      # ← Low band antenna (different!)
        antenna_pattern_lo='38.901',
        order="hi_lo",
    )

# Drop UEs
sim.drop_ues(num_ue=500, layout='box', box_pad_m=250, seed=7)
sim.cells_chunk = 48   # try 32/48/64 depending on memory
sim.ue_chunk = 500  # or 500/2000; whatever fits

In [ ]:
RSRP_dBm, cells_meta = sim.compute()

In [ ]:
RSRP_dBm.shape



In [ ]:
rows = rsrp_rows_as_dicts(RSRP_dBm, cells_meta, threshold_dbm=-124.0, label_mode="name")

In [ ]:
cells_meta

In [ ]:
highest_server = []
for i in range(500):
    highest_server.append(sorted(RSRP_dBm[i], reverse=True)[0])

In [ ]:
sorted(RSRP_dBm[150], reverse=True)

In [ ]:
rows[-3]

In [ ]:
arr = RSRP_dBm

In [ ]:
# Get indices of the max in each row
max_idx = np.argmax(arr, axis=1)

# Gather the max values
row_indices = np.arange(arr.shape[0])
max_vals = arr[row_indices, max_idx]

# Replace the max with -inf temporarily, then find 2nd max
arr_copy = arr.copy()
arr_copy[row_indices, max_idx] = -np.inf
second_max_vals = np.max(arr_copy, axis=1)

# dB difference
db_diff = max_vals - second_max_vals

print(db_diff.shape)  # (1000,)
print(db_diff[:10])   # show first 10 differences